In [1]:
# Importing necessary packages
import os
import numpy as np
import pandas as pd
import logging
import warnings
import joblib
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import csv

from sklearn.metrics import classification_report, roc_curve, accuracy_score, confusion_matrix, roc_auc_score, precision_score, recall_score, f1_score, precision_recall_curve
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import RobustScaler
import xgboost as xgb
import matplotlib.pyplot as plt


warnings.filterwarnings('ignore')

# Setting up logging with a FileHandler
log_file_path = 'logger_log.txt'
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', datefmt='%Y-%m-%d %H:%M:%S')

file_handler = logging.FileHandler(log_file_path)
file_handler.setLevel(logging.INFO)
file_handler.setFormatter(logging.Formatter('%(asctime)s - %(levelname)s - %(message)s'))

logger = logging.getLogger(__name__)
logger.addHandler(file_handler)

In [2]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float32)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df


In [3]:
# Reading the dataset
dataset_path = "/kaggle/input/us-stock-market-2020-to-2024/US Stock Market Dataset.csv"

try:
    # Attempt to read the dataset
    df = pd.read_csv(dataset_path)
    logger.info(f"Dataset loaded successfully from {dataset_path}")
except FileNotFoundError:
    logger.error("Error: Dataset file not found. Please provide the correct file path.")
except Exception as e:
    logger.error(f"An error occurred: {e}")
    
df

,Unnamed: 0,Date,Natural_Gas_Price,Natural_Gas_Vol.,Crude_oil_Price,Crude_oil_Vol.,Copper_Price,Copper_Vol.,Bitcoin_Price,Bitcoin_Vol.,...,Berkshire_Price,Berkshire_Vol.,Netflix_Price,Netflix_Vol.,Amazon_Price,Amazon_Vol.,Meta_Price,Meta_Vol.,Gold_Price,Gold_Vol.
0,0,2/2/2024,2.079,NaN,72.28,NaN,3.8215,NaN,"43,194.70",42650,...,"5,89,498",10580,564.64,4030000,171.81,117220000,474.99,84710000,"2,053.70",NaN
1,1,1/2/2024,2.050,161340.0,73.82,577940.0,3.8535,NaN,"43,081.40",47690,...,"5,81,600",9780,567.51,3150000,159.28,66360000,394.78,25140000,"2,071.10",260920.0
2,2,31-01-2024,2.100,142860.0,75.85,344490.0,3.9060,NaN,"42,580.50",56480,...,"5,78,020",9720,564.11,4830000,155.20,49690000,390.14,20010000,"2,067.40",238370.0
3,3,30-01-2024,2.077,139750.0,77.82,347240.0,3.9110,NaN,"42,946.20",55130,...,"5,84,680",9750,562.85,6120000,159.00,42290000,400.06,18610000,"2,050.90",214590.0
4,4,29-01-2024,2.490,3590.0,76.78,331930.0,3.8790,NaN,"43,299.80",45230,...,"5,78,800",13850,575.79,6880000,161.26,42840000,401.02,17790000,"2,034.90",1780.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1008,1008,8/1/2020,2.141,242560.0,59.61,1210000.0,2.8120,77700.0,"8,059.60",1190000,...,"3,39,188",190,339.26,7110000,94.60,70240000,215.22,13490000,"1,560.20",813410.0
1009,1009,7/1/2020,2.162,163010.0,62.70,582650.0,2.7935,59470.0,"8,155.70",1010000,...,"3,38,901",280,330.75,4740000,95.34,82680000,213.06,15110000,"1,574.30",435870.0
1010,1010,6/1/2020,2.135,154680.0,63.27,724240.0,2.7900,59570.0,"7,759.10",786750,...,"3,40,210",280,335.83,5670000,95.14,81310000,212.60,17070000,"1,568.80",558970.0
1011,1011,3/1/2020,2.130,144670.0,63.05,885860.0,2.7870,74750.0,"7,343.10",936290,...,"3,39,155",220,325.90,3800000,93.75,75330000,208.67,11200000,"1,552.40",436740.0


In [4]:
# Rename the first column
df = df.rename(columns={df.columns[0]: 'date_index'})

df = df[['date_index','Natural_Gas_Price']]
df['prev_index'] = df['date_index'] + 1
df.head(5)

,date_index,Natural_Gas_Price,prev_index
0,0,2.079,1
1,1,2.050,2
2,2,2.100,3
3,3,2.077,4
4,4,2.490,5


In [5]:
df['prev_index'] = df['date_index'] + 2
df.head(5)

,date_index,Natural_Gas_Price,prev_index
0,0,2.079,2
1,1,2.050,3
2,2,2.100,4
3,3,2.077,5
4,4,2.490,6


In [6]:
y = 1
df['prev_index'] = df['date_index'] + y
df = pd.merge(df, df, left_on='date_index', right_on='prev_index', how='inner', suffixes=(y,''))
df = df.drop(['date_index1','prev_index1','prev_index'],axis=1)
df

,Natural_Gas_Price1,date_index,Natural_Gas_Price
0,2.050,0,2.079
1,2.100,1,2.050
2,2.077,2,2.100
3,2.490,3,2.077
4,2.712,4,2.490
...,...,...,...
1007,2.141,1007,2.166
1008,2.162,1008,2.141
1009,2.135,1009,2.162
1010,2.130,1010,2.135


In [7]:
y = 1
#merged_df['prev_index'] = merged_df['date_index'] + y
#merged_df = pd.merge(df,merged_df, left_on='prev_index', right_on='date_index', how='inner', suffixes=('_src','_dst'))

# Define the specific suffix you want to rename
#old_suffix = '_src'
#new_suffix = ''
#columns_to_rename = [col for col in merged_df.columns if col.endswith(old_suffix)]
#merged_df = merged_df.rename(columns={col: col.replace(old_suffix, new_suffix) for col in columns_to_rename})

df

,Natural_Gas_Price1,date_index,Natural_Gas_Price
0,2.050,0,2.079
1,2.100,1,2.050
2,2.077,2,2.100
3,2.490,3,2.077
4,2.712,4,2.490
...,...,...,...
1007,2.141,1007,2.166
1008,2.162,1008,2.141
1009,2.135,1009,2.162
1010,2.130,1010,2.135


In [8]:
def create_df_panel(index_column_name,df,start,end,back_window):
    
    counter = 0
        
    for x in range(start,(end+1),1):    
        print("==================================")
        print("Round:",x)
        counter_in = 0    
        
        df1 = df[df[index_column_name] == x]
                           
        for x1 in range(x+1,(x+back_window+1),1):
            print("Window:",x1)
            
            df2 = df[df[index_column_name] == x1]
            df2 = df2.add_suffix('_'+str(x1))

            if counter_in == 0:
                final =  pd.concat([df1.reset_index(),df2.reset_index()], axis=1)
                counter_in = 1
            else:
                final =  pd.concat([final.reset_index(),df2.reset_index()], axis=1)
            
        if counter == 0:
            counter = 1
            final_df = final
        else:
            final_df = pd.concat([final_df, final], ignore_index=True)
        
    return final_df
            
index_column_name = 'date_index'
start = 0
end = 0
back_window = 3
#final = create_df_panel(index_column_name,df,start,end,back_window)
#print(final.columns)
#final

In [9]:
import pandas as pd

# Create two example DataFrames
df1 = pd.DataFrame({'A': [1, 2, 3], 'B': [4, 5, 6]})
df2 = pd.DataFrame({'C': [7, 8, 9], 'D': [10, 11, 12]})

# Concatenate along columns (axis=1)
result = pd.concat([df1, df2], axis=1)

print(result)

   A  B  C   D
0  1  4  7  10
1  2  5  8  11
2  3  6  9  12


In [10]:
# Save log messages to a file
# File handling: close the FileHandler
file_handler.close()
logger.removeHandler(file_handler)